In [2]:
!pip install spacy

In [3]:
!pip install tensorflow-hub
import tensorflow_hub as hub
import datetime, os

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import preprocessing
from keras.losses import BinaryCrossentropy

pd.set_option('max_columns', 500)

In [4]:
#read in and process data
df = pd.read_csv('/home/aschharwood/notebooks/covid/feature_target_v1_county_nyt_cdc_moa_tweets_gdelt_apple.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)

#df.info(max_cols=500)

mean = df._get_numeric_data().mean()

#fill empty mobility data with column mean
df.fillna(mean, inplace=True)

#df.head()

df['COUNTYFP'] = df['COUNTYFP'].astype('string')

#df.describe()

import numpy as np

#create binary target
df['always_binary'] = np.where(df['ALWAYS']>.50, 1, 0)

df['always_binary'].value_counts()
df.fillna(df._get_numeric_data().mean(), inplace=True)

In [5]:
#clean up text data
df['text_tokens_str'] = df['text_tokens_str'].replace(r'\n',' ', regex=True)
df['High'] = df['High'].replace(r'\n',' ', regex=True)
df['Low'] = df['Low'].replace(r'\n',' ', regex=True)

In [6]:
#discard unneeded mask wearing columns
df.drop(['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 'FIPS'], axis=1, inplace=True)

In [7]:
y = df['always_binary']
X = df.drop('always_binary', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [8]:
#grab spacy's language model
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz


     |████████████████████████████████| 12.0 MB 1.4 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=d7b266569a50e657948703fd76cee73fa30e23c65fa82975b5fa705357acd886
  Stored in directory: /home/aschharwood/.cache/pip/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm


In [9]:
#import and load spacy language model
import en_core_web_sm

In [10]:
nlp = en_core_web_sm.load(disable=['tagger', 'parser', 'ner'])

In [11]:
#help text tokenization function
def tokenizer(text, nlp):

    token_list = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False and token.is_punct==False and token.like_url==False:
            if token.text != ' ':
                token_list.append((token.lemma_).lower())
    str_tokens = ' '.join(token_list)
    return str_tokens

## Googles News Stacked

- This model combines numeric inputs from CDC Social Vulnerability Index, Measure of America Youth Disconnection Index, and Apple mobility data at the county-level, with county-level geolocated tweets and state geotagged Covid data. It doesn't work very at the moment, but when I run each part of the model independently, I'm starting to see results. So I have still have some debugging to do. Nevertheless, this represents an initial attempt at building a stacked neural network architecutre that combines text data with word embeddings and numeric metadata.

- The target for this model is binary classification about whether more or less than 50 percent of the population for each county wear's a mask. It's derived from the New York Times July 2020 survey into mask wearing.

In [12]:
y = df['always_binary']
X = df.drop('always_binary', axis=1)

In [13]:
cats = pd.get_dummies(X[['COUNTYFP', 'ST_ABBR']])

In [14]:
X = pd.concat([X, cats], axis=1)

In [15]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

In [16]:
X2_train = X_train.drop(['COUNTYFP', 'ST_ABBR', 'text_tokens_str', 'High', 'Low'], axis=1)

X2_test = X_test.drop(['COUNTYFP', 'ST_ABBR', 'text_tokens_str', 'High', 'Low'], axis=1)

#X2_train = X_train.drop(['COUNTYFP', 'ST_ABBR'], axis=1)
scaler = MinMaxScaler()
X2_train = scaler.fit_transform(X2_train)


X2_test = scaler.transform(X2_test)

In [17]:
#download and set word vector pretrained model
#embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
#hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

embedding = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'

print('Google news model downlownded')

Google news model downlownded


In [18]:
X_News_High_train = X_train['High'].apply(lambda x: tokenizer(x, nlp))
X_News_High_test = X_test['High'].apply(lambda x: tokenizer(x, nlp))

In [19]:
X_News_Low_train = X_train['Low'].apply(lambda x: tokenizer(x, nlp))
X_News_Low_test = X_test['Low'].apply(lambda x: tokenizer(x, nlp))

In [20]:
X_tweet_train = X_train['text_tokens_str'].apply(lambda x: tokenizer(x, nlp))
X_tweet_test = X_test['text_tokens_str'].apply(lambda x: tokenizer(x, nlp))

In [21]:
X2_train.shape

(2513, 214)

In [28]:
tf.keras.layers.BatchNormalization()

In [29]:


#numeric input
num_input = Input(shape=(214,))
dense_layer_1_num = Dense(10, activation='relu')(num_input)
batch_out_num = tf.keras.layers.BatchNormalization()(dense_layer_1_num) 
num_output = Dense(10, activation='relu')(batch_out_num)

#tweets low
tweets = Input(shape=[], dtype=tf.string)
hub_layer_tw = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)(tweets)
dense_1_tw = Dense(16, activation='relu')(hub_layer_tw)
dense_2_tw = Dense(8, activation='relu')(dense_1_tw)
batch_out_tweets = tf.keras.layers.BatchNormalization()(dense_2_tw) 
tw_output = Dense(4, activation='relu')(batch_out_tweets)
#tw_output = Dropout(0.5)(dense_3_tw)

#news low
news_low = Input(shape=[], dtype=tf.string)
hub_layer_nl = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)(news_low)
dense_1_nl = Dense(16, activation='relu')(hub_layer_nl)
dense_2_nl = Dense(8, activation='relu')(dense_1_nl)
batch_out_nl = tf.keras.layers.BatchNormalization()(dense_2_nl) 
nl_output = Dense(4, activation='relu')(batch_out_nl)
#nl_output = Dropout(0.5)(dense_3_nl)

#news high
news_high = Input(shape=[], dtype=tf.string)
hub_layer_news_high = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)(news_high)
dense_1_news_high = Dense(16, activation='relu')(hub_layer_news_high)
dense_2_nh = Dense(8, activation='relu')(dense_1_news_high)
batch_out_nh = tf.keras.layers.BatchNormalization()(dense_2_nh) 
nh_output = Dense(4, activation='relu')(batch_out_nh)
#nh_output = Dropout(0.5)(dense_3_nh)


#concat layer takes output layers from tweet and num models, which can be passed to other models
concat_layer = Concatenate()([num_output, tw_output, nl_output, nh_output])
dense_1_cl = Dense(100, activation='relu')(concat_layer)
dense_2_cl = Dense(80, activation='relu')(dense_1_cl)
dense_3_cl = Dense(40, activation='relu')(dense_2_cl)
dense_4_cl = Dense(40, activation='relu')(dense_3_cl)
output = Dense(1, activation='softmax')(dense_4_cl)
model = Model(inputs=[num_input, tweets, news_low, news_high], outputs=output)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [30]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

validation_data=([X2_test, X_tweet_test, X_News_Low_test, X_News_High_test], y_test)

print('training model')
history = model.fit(x=[X2_train, X_tweet_train, X_News_Low_train, X_News_High_train], y=y_train, epochs=5, verbose=1, validation_data=validation_data)
print('model training complete')

model defined and compiled
training model
Epoch 1/5
79/79 [==============================] - 34s 434ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 2/5
79/79 [==============================] - 34s 433ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 3/5
79/79 [==============================] - 35s 437ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 4/5
79/79 [==============================] - 34s 436ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 5/5
79/79 [==============================] - 35s 440ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
model training complete


# Model - Num Only

- this model performs reasonably well with just the static socioeconomic and demographic data. 

In [84]:
num_input = Input(shape=(214,))
dense_layer_1_num = Dense(10, activation='relu')(num_input)
num_output = Dense(1, activation='relu')(dense_layer_1_num)
model = Model(inputs=[num_input], outputs=num_output)


optimizer = tf.keras.optimizers.Adam(lr=.01)
model.compile(optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')


validation_data=([X2_test], y_test)
x=[X2_train]

print('training model')
history = model.fit(x=x, y=y_train, epochs=10, verbose=1, validation_data=validation_data)
print('model training complete')

model defined and compiled
training model
Epoch 1/10
89/89 [==============================] - 0s 2ms/step - loss: 0.6055 - accuracy: 0.7060 - val_loss: 0.5720 - val_accuracy: 0.7905
Epoch 2/10
89/89 [==============================] - 0s 938us/step - loss: 0.5459 - accuracy: 0.8051 - val_loss: 0.5537 - val_accuracy: 0.7905
Epoch 3/10
89/89 [==============================] - 0s 905us/step - loss: 0.5356 - accuracy: 0.8097 - val_loss: 0.5767 - val_accuracy: 0.7905
Epoch 4/10
89/89 [==============================] - 0s 963us/step - loss: 0.5393 - accuracy: 0.8062 - val_loss: 0.5984 - val_accuracy: 0.7968
Epoch 5/10
89/89 [==============================] - 0s 930us/step - loss: 0.5263 - accuracy: 0.8168 - val_loss: 0.5425 - val_accuracy: 0.8063
Epoch 6/10
89/89 [==============================] - 0s 1ms/step - loss: 0.5281 - accuracy: 0.8146 - val_loss: 0.5441 - val_accuracy: 0.7714
Epoch 7/10
89/89 [==============================] - 0s 939us/step - loss: 0.5292 - accuracy: 0.8065 - val_loss

In [70]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 214)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2150      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 2,161
Trainable params: 2,161
Non-trainable params: 0
_________________________________________________________________


# Model Debug - 


In [24]:
embedding = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'
tweets = Input(shape=[], dtype=tf.string)
hub_layer_tw = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)(tweets)
dense_1_tw = Dense(16, activation='relu')(hub_layer_tw)
dense_2_tw = Dense(8, activation='relu')(dense_1_tw)
tw_output = Dense(1, activation='relu')(dense_2_tw)

model = Model(inputs=[tweets], outputs=tw_output)


optimizer = tf.keras.optimizers.Adam(lr=.01)
model.compile(optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')


validation_data=([X_tweet_test], y_test)
x=[X_tweet_train]

print('training model')
history = model.fit(x=x, y=y_train, epochs=10, verbose=1, validation_data=validation_data)
print('model training complete')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



model defined and compiled
training model
Epoch 1/10
79/79 [==============================] - 16s 203ms/step - loss: 0.6410 - accuracy: 0.6407 - val_loss: 0.6543 - val_accuracy: 0.6312
Epoch 2/10
79/79 [==============================] - 16s 201ms/step - loss: 0.5882 - accuracy: 0.6888 - val_loss: 0.7667 - val_accuracy: 0.6184
Epoch 3/10
79/79 [==============================] - 16s 201ms/step - loss: 0.5604 - accuracy: 0.7079 - val_loss: 0.8077 - val_accuracy: 0.5978
Epoch 4/10
79/79 [==============================] - 16s 201ms/step - loss: 0.5507 - accuracy: 0.7163 - val_loss: 0.8892 - val_accuracy: 0.6025
Epoch 5/10
79/79 [==============================] - 16s 202ms/step - loss: 0.5479 - accuracy: 0.7191 - val_loss: 0.9460 - val_accuracy: 0.6010
Epoch 6/10
79/79 [==============================] - 16s 201ms/step - loss: 0.5468 - accuracy: 0.7199 - val_loss: 1.0792 - val_accuracy: 0.6200
Epoch 7/10
79/79 [==============================] - 16s 203ms/step - loss: 0.5465 - accuracy: 0.7203

## Model debug - stacke

In [25]:
embedding = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'


#numeric input
num_input = Input(shape=(214,))
dense_layer_1_num = Dense(10, activation='relu')(num_input)
num_output = Dense(10, activation='relu')(dense_layer_1_num)

#tweets low
tweets = Input(shape=[], dtype=tf.string)
hub_layer_tw = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)(tweets)
dense_1_tw = Dense(100, activation='relu')(hub_layer_tw)
tw_output = Dense(32, activation='relu')(dense_1_tw)
#tw_output = Dropout(0.5)(dense_3_tw)


#concat layer takes output layers from tweet and num models, which can be passed to other models
concat_layer = Concatenate()([num_output, tw_output])

output = Dense(1, activation='softmax')(concat_layer)
model = Model(inputs=[num_input, tweets], outputs=output)



optimizer = tf.keras.optimizers.Adam(lr=.1)
model.compile(optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')


validation_data=([X2_test, X_tweet_test], y_test)
x=[X2_train, X_tweet_train]

print('training model')
history = model.fit(x=x, y=y_train, epochs=5, verbose=1, validation_data=validation_data)
print('model training complete')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



model defined and compiled
training model
Epoch 1/5
79/79 [==============================] - 16s 202ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 2/5
79/79 [==============================] - 16s 201ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 3/5
79/79 [==============================] - 16s 201ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 4/5
79/79 [==============================] - 16s 200ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
Epoch 5/5
79/79 [==============================] - 16s 200ms/step - loss: 0.8218 - accuracy: 0.4914 - val_loss: 0.8220 - val_accuracy: 0.4913
model training complete


In [ ]:
#inital preprocessing for text using GloVe. Discarded in favor of a covid-pretrained dataset
# def preprocess_text(sen):

#     # Remove punctuations and numbers
#     sentence = re.sub('[^a-zA-Z]', ' ', sen)

#     # Single character removal
#     sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

#     # Removing multiple spaces
#     sentence = re.sub(r'\s+', ' ', sentence)

#     return sentence



# X1_train = []
# sentences = list(X_train['text_tokens_str'])
# for sen in sentences:
#     X1_train.append(preprocess_text(sen))
    
# X1_test = []
# sentences = list(X_test["text_tokens_str"])
# for sen in sentences:
#     X1_test.append(preprocess_text(sen))
    
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X1_train)

# X1_train = tokenizer.texts_to_sequences(X1_train)
# X1_test = tokenizer.texts_to_sequences(X1_test)

# vocab_size = len(tokenizer.word_index) + 1

# maxlen = 200

# X1_train = pad_sequences(X1_train, padding='post', maxlen=maxlen)
# X1_test = pad_sequences(X1_test, padding='post', maxlen=maxlen)

# from numpy import array
# from numpy import asarray
# from numpy import zeros

# embeddings_dictionary = dict()

# glove_file = open('/home/aschharwood/notebooks/covid/notebooks/glove_tweets/glove.twitter.27B.100d.txt')

# for line in glove_file:
#     records = line.split()
#     word = records[0]
#     vector_dimensions = asarray(records[1:], dtype='float32')
#     embeddings_dictionary[word] = vector_dimensions
# glove_file.close()

# embedding_matrix = zeros((vocab_size, 100))
# for word, index in tokenizer.word_index.items():
#     embedding_vector = embeddings_dictionary.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[index] = embedding_vector

In [ ]:
# X2_train = X_train.drop(['COUNTYFP', 'ST_ABBR', 'text_tokens_str', 'High', 'Low'], axis=1)

# X2_test = X_test.drop(['COUNTYFP', 'ST_ABBR', 'text_tokens_str', 'High', 'Low'], axis=1)

# #X2_train = X_train.drop(['COUNTYFP', 'ST_ABBR'], axis=1)
# scaler = MinMaxScaler()
# X2_train = scaler.fit_transform(X2_train)


# X2_test = scaler.transform(X2_test)

In [ ]:
# tweet_input = Input(shape=(maxlen,))
# num_input = Input(shape=(163,))

In [ ]:
# #text model
# embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(tweet_input)
# LSTM_Layer_1 = LSTM(128)(embedding_layer)

# #num model
# dense_layer_1 = Dense(10, activation='relu')(num_input)
# dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)

In [ ]:
# #concat layer takes output layers from tweet and num models, which can be passed to other models
# concat_layer = Concatenate()([LSTM_Layer_1, dense_layer_2])
# dense_layer_3 = Dense(10, activation='relu')(concat_layer)
# output = Dense(1, activation='softmax')(dense_layer_3)
# model = Model(inputs=[tweet_input, num_input], outputs=output)

In [ ]:
# model.compile(optimizer='adam',
#              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              metrics=['accuracy'])
# print('model defined and compiled')

In [ ]:
# print('training model')
# history = model.fit(x=[X1_train, X2_train], y=y_train, epochs=3, verbose=1, validation_data=([X1_test, X2_test], y_test))
# print('model training complete')